In [72]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [73]:
from extract.polocm import AP, Hypothesis, POLOCM
from traces import *
from convertor import TopoConvertor
from observation import PartialOrderedActionObservation, ObservedPartialOrderTraceList
from extract.model import *
import pulp as pl
from tabulate import tabulate

In [74]:
objects = {
    "A": PlanningObject("unknown", "A"),
    "B": PlanningObject("unknown", "B"),
    "C": PlanningObject("unknown", "C"),
}

actions = {
    "unstackAB": Action("unstack", [objects["A"], objects["B"]]),
    "unstackBC": Action("unstack", [objects["B"], objects["C"]]),
    "unstackCB": Action("unstack", [objects["C"], objects["B"]]),
    "unstackBA": Action("unstack", [objects["B"], objects["A"]]),
    "unstackAC": Action("unstack", [objects["A"], objects["C"]]),
    "unstackCA": Action("unstack", [objects["C"], objects["A"]]),
    "stackAB": Action("stack", [objects["A"], objects["B"]]),
    "stackBA": Action("stack", [objects["B"], objects["A"]]),
    "stackAC": Action("stack", [objects["A"], objects["C"]]),
    "stackCA": Action("stack", [objects["C"], objects["A"]]),
    "stackBC": Action("stack", [objects["B"], objects["C"]]),
    "stackCB": Action("stack", [objects["C"], objects["B"]]),
    "putdownA": Action("putdown", [objects["A"]]),
    "putdownB": Action("putdown", [objects["B"]]),
    "putdownC": Action("putdown", [objects["C"]]),
    "pickupA": Action("pickup", [objects["A"]]),
    "pickupB": Action("pickup", [objects["B"]]),
    "pickupC": Action("pickup", [objects["C"]]),
}

traces = TraceList(
    [
        Trace(
            [
                Step(State({}), actions["unstackAB"], 0),
                Step(State({}), actions["putdownA"], 1),
                Step(State({}), actions["pickupB"], 2),
                Step(State({}), actions["stackBC"], 3),
                Step(State({}), actions["unstackBC"], 4),
                Step(State({}), actions["stackBC"], 5),
                Step(State({}), actions["pickupA"], 6),
                Step(State({}), actions["stackAB"], 7),
            ]
        ),
        Trace(
            [
                Step(State({}),actions['unstackCB'], 0),
                Step(State({}),actions['putdownC'], 1),
                Step(State({}),actions['unstackBA'], 2),
                Step(State({}),actions['putdownB'], 3),
                Step(State({}),actions['pickupB'], 4),
                Step(State({}),actions['stackBC'], 5),
                Step(State({}),actions['pickupA'], 6),
                Step(State({}),actions['stackAB'], 7),

            ]
        ),
        Trace(
            [
                Step(State({}), actions["unstackCB"], 0),
                Step(State({}), actions["stackCA"], 1),
                Step(State({}), actions["pickupB"], 2),
                Step(State({}), actions["putdownB"], 3),
                Step(State({}), actions["pickupB"], 4),
                Step(State({}), actions["stackBC"], 5),
            ]
        ),
        
    ]
)


In [75]:
convertor = TopoConvertor("flex", True, True, 1)
po_tracelist = traces.topo(convertor, 0.2)
print(tabulate(po_tracelist[0].comparable_matrix, headers='keys', tablefmt="fancy_grid"))
obs_tracelist = po_tracelist.tokenize(PartialOrderedActionObservation, ObservedPartialOrderTraceList)

╒════════════╤══════════════╤══════════════╤═════════════╤════════════╤══════════════╤════════════╤═════════════╤════════════╕
│            │   unstack[0] │   putdown[1] │   pickup[2] │   stack[3] │   unstack[4] │   stack[5] │   pickup[6] │   stack[7] │
╞════════════╪══════════════╪══════════════╪═════════════╪════════════╪══════════════╪════════════╪═════════════╪════════════╡
│ unstack[0] │          nan │          nan │           1 │          1 │            1 │          1 │           1 │          1 │
├────────────┼──────────────┼──────────────┼─────────────┼────────────┼──────────────┼────────────┼─────────────┼────────────┤
│ putdown[1] │          nan │          nan │           1 │          1 │            1 │          1 │           1 │          1 │
├────────────┼──────────────┼──────────────┼─────────────┼────────────┼──────────────┼────────────┼─────────────┼────────────┤
│ pickup[2]  │          nan │          nan │         nan │        nan │            1 │          1 │           1

In [76]:
debug = {
    'pstep6': True,
    '2step0': True,
    'step7':True
}
model,AP, _ = POLOCM(obs_tracelist, debug=debug)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/uto/miniconda3/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/c9c3ee221d0d4d119e7f5688116c517f-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/c9c3ee221d0d4d119e7f5688116c517f-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1867 COLUMNS
At line 6818 RHS
At line 8681 BOUNDS
At line 9104 ENDATA
Problem MODEL has 1862 rows, 422 columns and 4068 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 13 - 0.01 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 437 strengthened rows, 128 substitutions
Cgl0003I 4 fixed, 0 tightened bounds, 131 strengthened rows, 30 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 59 strengthened rows, 1 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 15 strengthened rows, 12 substitutions
Cgl0003I 0 fixed, 0 tightened bou

## Solution FO matrix

### Trace 0

#### Obj zero

╒════════════╤═════════════╤════════════╤══════════════╤════════════╤══════════════╤══════════════╤════════════╤═════════════╕
│            │   [2]pickup │   [5]stack │   [1]putdown │   [7]stack │   [0]unstack │   [4]unstack │   [3]stack │   [6]pickup │
╞════════════╪═════════════╪════════════╪══════════════╪════════════╪══════════════╪══════════════╪════════════╪═════════════╡
│ [2]pickup  │         nan │          0 │            0 │          0 │            0 │            0 │          1 │           0 │
├────────────┼─────────────┼────────────┼──────────────┼────────────┼──────────────┼──────────────┼────────────┼─────────────┤
│ [5]stack   │           0 │        nan │            0 │          0 │            0 │            0 │          0 │           1 │
├────────────┼─────────────┼────────────┼──────────────┼────────────┼──────────────┼──────────────┼────────────┼─────────────┤
│ [1]putdown │           1 │          0 │          nan │          0 │            0 │            0 │          0 

...

Checking candidate set *{stack.2, stack.1, pickup.1}* of **Sort.2** for well formedness and Validity

,stack.2,stack.1,pickup.1
stack.2,0,0,0
stack.1,3,0,0
pickup.1,0,5,0


0 1
0 2
1 2
This subset is well-formed.
This subset is valid.
Adding this subset {stack.2, stack.1, pickup.1} to the locm2 transition set.
Hole that is covered now:
[stack.1, pickup.1]
[{putdown.1, stack.1, unstack.1}, {stack.2, stack.1, pickup.1}]

Removed redundancy transition set list
[{putdown.1, stack.1, unstack.1}, {stack.2, stack.1, pickup.1}]


#### Final transition set list

[{putdown.1, stack.1, unstack.1}, {stack.2, stack.1, pickup.1}, {stack.1, unstack.1, unstack.2, putdown.1, stack.2, pickup.1}]
TS_overall: 
 [{F0S0: {zero zero: [unstack.0, putdown.0, pickup.0, stack.0, unstack.0, stack.0, pickup.0, stack.0]}, F0S1: {unknown B: [stack.1, unstack.1, stack.1], unknown C: [], unknown A: [unstack.1, putdown.1, stack.1]}, F1S1: {unknown B: [pickup.1, stack.1, stack.1, stack.2], unknown C: [stack.2, stack.2], unknown A: [pickup.1, stack.1]}, F2S1: {unknown B: [unstack.2, pickup.1, stack.1, unstack.1, stack.1, stack.2], unknown C: [stack.2, unstack.2, stack.2], unknown A: [unstack.1, putdown.1, pickup.1, stack.1]}}, {F0S0: {zero zero: [unstack.0, putdown.0, unstack.0, putdown.0, pickup.0, stack.0, pickup.0, stack.0]}, F0S1: {unknown A: [stack.1], unknown B: [unstack.1, putdown.1, stack.1], unknown C: [unstack.1, putdown.1]}, F1S1: {unknown A: [pickup.1, stack.1], unknown B: [pickup.1, stack.1, stack.2], unknown C: [stack.2]}, F2S1: {unknown A: [unstack.2, pic

In [77]:
model.to_pddl('polocm')

unstack
{(F2S1state0 sort1 sort1), (F0S1state2 sort1 sort1), (F2S1state2 sort1), (F0S0state0 sort0)}
[x0 (sort0), x1 (sort1), x2 (sort1)]
(F2S1state0 sort1 sort1)
[2, 1]
(F0S1state2 sort1 sort1)
[1, 2]
(F2S1state2 sort1)
[1]
(F0S0state0 sort0)
[0]
pickup
{(F1S1state3 sort1), (F2S1state2 sort1), (F0S0state0 sort0)}
[x0 (sort0), x1 (sort1)]
(F1S1state3 sort1)
[1]
(F2S1state2 sort1)
[1]
(F0S0state0 sort0)
[0]
putdown
{(F0S1state1 sort1), (F0S0state1 sort0), (F2S1state1 sort1)}
[x0 (sort0), x1 (sort1)]
(F0S1state1 sort1)
[1]
(F0S0state1 sort0)
[0]
(F2S1state1 sort1)
[1]
stack
{(F0S1state1 sort1), (F2S1state2 sort1), (F1S1state2 sort1), (F0S0state1 sort0), (F2S1state1 sort1), (F1S1state0 sort1)}
[x0 (sort0), x1 (sort1), x2 (sort1)]
(F0S1state1 sort1)
[1]
(F2S1state2 sort1)
[2]
(F1S1state2 sort1)
[1]
(F0S0state1 sort0)
[0]
(F2S1state1 sort1)
[1]
(F1S1state0 sort1)
[2]
